In [2]:
import requests

### Query Params for eBay's "Completed Listings" API Endpoint

In [3]:
API_KEY = 'RobertBo-cinemaro-PRD-171ca9e35-af8bdcbb' # Enter your API Key/"App ID" Here. Mine was 40 chars long.

In [4]:
# FILM_CAMS = '15230'
ELEC_GUITARS = '33034' # Category code for electric guitars on eBay.

In [5]:
COUNTRY = 'EBAY-US' # USA Marketplace only. API seems to start returning stuff from other markets eventually anyhow

In [6]:
USED = '3000' # Just the code for item condition "used". This is the focus of this project.

In [7]:
ITEM_FILTER_0 = f'itemFilter(0).name=Condition&itemFilter(0).value={USED}' # Only used guitars
ITEM_FILTER_1 = f'itemFilter(1).name=HideDuplicateItems&itemFilter(1).value=true' # No duplicate listings
ITEM_FILTER_2 = f'itemFilter(2).name=MinPrice&itemFilter(2).value=249' # Only items that sold for > this value
ITEM_FILTER_3 = f'itemFilter(3).name=MaxQuantity&itemFilter(3).value=1' # No lots or batch sales. One axe at a time
ITEM_FILTER_4 = f'itemFilter(4).name=SoldItemsOnly&itemFilter(4).value=true' # Only looking at listings that sold.
ITEM_FILTER_5 = f'itemFilter(5).name=ListingType&itemFilter(5).value=Auction' # Only looking at auctions.

In [8]:
FIND_COMPLETED = 'findCompletedItems' # This is the eBay API endpoint service we'll be querying.

### Actual Query

In [9]:
PAGE = str(3) # This value is currently static but will be looped when it's time to persist some data

Here we go:

In [ ]:
# r = requests.get(f'https://svcs.ebay.com/services/search/FindingService/v1?'
#                  f'OPERATION-NAME={FIND_COMPLETED}&'
#                  f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
#                  f'RESPONSE-DATA-FORMAT=JSON&' # This value can be altered if you're not into JSON responses
#                  f'REST-PAYLOAD&'
#                  f'GLOBAL-ID={COUNTRY}&' # seems to prioritize the value you enter but returns other stuff too
#                  f'categoryId={ELEC_GUITARS}&' # Product category goes here
#                  f'descriptionSearch=true&' # More verbose responses
#                  f'{ITEM_FILTER_0}&'
#                  f'{ITEM_FILTER_1}&'
#                  f'{ITEM_FILTER_2}&' # Filters defined previously
#                  f'{ITEM_FILTER_3}&'
#                  f'{ITEM_FILTER_4}&'
#                  f'{ITEM_FILTER_5}&'
#                  f'paginationInput.pageNumber={PAGE}&' # value to be looped through when collecting lotsa data
#                  f'outputSelector=PictureURLLarge') # Why not grab the thumbnail URLs too. Could be cool

In [ ]:
listo = r.json()

In [ ]:
listo['findCompletedItemsResponse'][0]['searchResult'][0]['@count'] # Number of listings returned by this query

In [ ]:
item = listo['findCompletedItemsResponse'][0]['searchResult'][0]['item'][98] # Inspecting the last item of response
item

In [10]:
ITEM_ID = item['itemId'][0] # Just grabbing some item's ID to check it on the item-specific API (soon)
ITEM_ID

NameError: name 'item' is not defined

### Which of these fields might be useful for a regression project?

* itemID, for passing to getSingleItem API


* NLP: Title
* NLP: SubTitle


* CNN: Thumbnail images


* scalar: Shipping Cost
* scalar: handlingTime
* scalar: Duration (derive from start/end cols)
* scalar: __convertedCurrentPrice__ - THIS IS THE INDEPENDENT VAR


* 1-hot: Payment method
* 1-hot: Seller Country
* 1-hot: ShippingType
* 1-hot: shipToLocations
* 1-hot: expeditedShipping
* 1-hot: oneDayShippingAvailable
* 1-hot: bestOfferEnabled
* 1-hot: Day of week (derive from start/end cols)
* 1-hot: Returns Accepted
* 1-hot: isMultiVariation (? What is that)
* 1-hot: topRatedListing (? what is that)

***

### What can we only get from the single-item API (below)?

* NLP: Description text
* NLP: Detailed __Condition__ Description


* Scalar: Length of description text (non-html tags)
* Scalar: Number of images on eBay normal
* Scalar: Year of manufacture
* Scalar: Seller.FeedbackScore
* Scalar: Seller.PositiveFeedbackPercent
* Scalar: ReturnsWithin number of days
* Scalar: len(ExcludeShipToLocation) aka how many countries excluded?


* 1-hot (or NLP?): Brand
* 1-hot (or NLP?): Model
* 1-hot: Right/left handed
* 1-hot: Body color
* 1-hot: Body material
* 1-hot: String Config (6-string, etc)
* 1-hot: Body type (Solid / hollow, etc)
* 1-hot: Soundboard Style (arch / flat top etc)
* 1-hot: Country of manufacture
* 1-hot: Case
* 1-hot: NewBestOffer
* 1-hot: AutoPay

### Item-Specific Queries, for More Features

Beware, you only get 5000 queries a day, so don't mash on this too hard.

In [ ]:
# r2 = requests.get('http://open.api.ebay.com/shopping?'
#                     f'callname=GetSingleItem&'
#                     f'responseencoding=JSON&'
#                     f'appid={API_KEY}&'
#                     f'siteid=0&' # USA Store
#                     f'version=967&' # What is this?
#                     f'ItemID={ITEM_ID}&' # Assigned above
#                     f'IncludeSelector=Details,ItemSpecifics,TextDescription')

Seems like there's some variability between items when it comes to item specifics field.

In [ ]:
r2.json()['Item']